`comparaison_env`

In [1]:
import pandas as pd
import glob
import os 
import sys
import trimesh
import numpy as np
import matplotlib.pyplot as plt
import random
import tqdm

sys.path.append(os.path.abspath("/home/pelissier/These-ATER/Papier_international3/Dataset"))
from utils import *
sys.path.append('/home/pelissier/These-ATER/Papier_international3/Code')  # Adjust the path based on the relative location
from utils_comparaison import *
from score_proximite import *

Variables globales

In [2]:
### MODELNET40 REMESHING ISO
ModelNet40_aligned_us = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned"
# Data des 12 caméras du Rendu
data_modelnet_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/circular_config_12_elevation_30_R22.csv")
# Path ok (avec limper + projection + bvs)
paths_bvs = read_paths_from_txt("/home/pelissier/These-ATER/Papier_international3/Dataset/paths_files/obj_SMPLER_files_ModelNet40_remeshing_iso_user-study-844-ok.txt"); print("Fichiers bvs de Modelnet40 :", len(paths_bvs))
dir_bvs = "/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/bvs_remeshing_iso"

##################################################################################################################################
### User study
# Path to the directory containing the csv files of the user study
dir_us = "/home/pelissier/These-ATER/Papier_internationale2/Validation/user_study/3D/post_traitement/csv_etude_Prolific"
# Data of camera poses in user study : i, j, theta, delta, X, Y, Z
data_us_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/cam_pose_rep_etude_arrondi.csv")
label_us_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/cam_rep_etude_label_arrondi.csv")
data_us_cam['label'] = list(label_us_cam['label'])
## Choix et bvs des 44 modeles 
dir_bvs_us = '/home/pelissier/These-ATER/Papier_internationale2/Validation/user_study/3D/post_traitement/csv_etude_Prolific/csv_etude_filtre/visualisation_filtre'
paths_bvs_us_csv = glob.glob(os.path.join(dir_bvs_us, "**", "*global_distribution_label.csv*"), recursive=True); print("Modeles de US : ",len(paths_bvs_us_csv))
# Path data folder of user study
dir_Data = "/home/pelissier/These-ATER/Papier_internationale2/Data"

##################################################################################################################################
# Correspondances entre les noms des modèles dans ModelNet10 et les noms des modèles dans l'User Study
match_ModelNet2US = {'aeroplane': 'A380', "chair":'chair107', 'bench': 'bench', 'dresser': 'cabinet', 'night_stand': 'cabinet', 'vase': 'vase', 'cup':'cup', 'car': 'carVasa'}
# Outputs tmp
# Path of user-study outputs folder in Dataset
dir_outputs_visu = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/visualisation_cams/"
dir_outputs_csv = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/csv_files/"

Fichiers bvs de Modelnet40 : 844
Modeles de US :  44


In [3]:
list_categorie_us = ['carVasa']
list_categroie_us_sym = ['carVasa']

## Contraintes : quelles caméras de lUS on considère ?

In [4]:
## Condition
## Toutes les caméras 
#j_cam_us_ok = [0,1,2,3,4]

## Que les cameras de la couronne j==1
j_cam_us_ok = [1]

## Que les cameras de la couronne j==1 ou j == 2
#j_cam_us_ok = [1,2]

## Cameras

In [5]:
## User study
# Coordonnées des caméras de l'étude utilateur
X_us = []; Y_us = []; Z_us = []; labels_us = []; I_us = []; J_us = []
for j in j_cam_us_ok:
    X_us += list(data_us_cam.loc[data_us_cam['j'] == j]['X_rep_etude'])
    Y_us += list(data_us_cam.loc[data_us_cam['j'] == j]['Y_rep_etude'])
    Z_us += list(data_us_cam.loc[data_us_cam['j'] == j]['Z_rep_etude'])
    I_us += list(data_us_cam.loc[data_us_cam['j'] == j]['i'])
    J_us += [j for _ in range(8)]
    labels_us += list(data_us_cam.loc[data_us_cam['j'] == j]['label']) 
cams_us = np.around(np.column_stack((X_us, Y_us, Z_us, np.array([1]*len(X_us)))),3)
R_sphere = list(data_us_cam['R'])[0]

## ModeleNet 
# Coordonnées des 12 caméras de ModelNet40 dans le repère de ModelNet40
X_modelnet = np.array(data_modelnet_cam['LocationX'][1:])
Y_modelnet = np.array(data_modelnet_cam['LocationY'][1:])
Z_modelnet = np.array(data_modelnet_cam['LocationZ'][1:])
cams_modelnet = np.column_stack((X_modelnet, Y_modelnet, Z_modelnet, np.array([1]*12)))
print("12 cams Modelnet dans repère Modelnet : \n", cams_modelnet)

12 cams Modelnet dans repère Modelnet : 
 [[ 0.         -2.2         1.1         1.        ]
 [ 1.1        -1.90525589  1.1         1.        ]
 [ 1.90525589 -1.1         1.1         1.        ]
 [ 2.2         0.          1.1         1.        ]
 [ 1.90525589  1.1         1.1         1.        ]
 [ 1.1         1.90525589  1.1         1.        ]
 [ 0.          2.2         1.1         1.        ]
 [-1.1         1.90525589  1.1         1.        ]
 [-1.90525589  1.1         1.1         1.        ]
 [-2.2         0.          1.1         1.        ]
 [-1.90525589 -1.1         1.1         1.        ]
 [-1.1        -1.90525589  1.1         1.        ]]


In [6]:
print(len(cams_us), "cams US", len(labels_us), "lables US")
print(str(len(cams_us))+" cams US considérées dans repère US : \n", cams_us, labels_us)

8 cams US 8 lables US
8 cams US considérées dans repère US : 
 [[ 1.56  1.56  0.    1.  ]
 [ 1.1   1.56  1.1   1.  ]
 [ 0.    1.56  1.56  1.  ]
 [-1.1   1.56  1.1   1.  ]
 [-1.56  1.56  0.    1.  ]
 [-1.1   1.56 -1.1   1.  ]
 [-0.    1.56 -1.56  1.  ]
 [ 1.1   1.56 -1.1   1.  ]] ['dessus_face', 'dessus_face_droit', 'dessus_profil_droit', 'dessus_arriere_droit', 'dessus_arriere', 'dessus_arriere_gauche', 'dessus_profil_gauche', 'dessus_face_gauche']


## Poids + BVS user study --> en fonction des caméras de l'us

In [7]:
all_bvs_us = {}
# Les POV qui nous intéressent sont ceux sur la couronne 'Milieu-Dessus', donc avec j == 1
for path_bvs_us_csv in paths_bvs_us_csv:
    name = os.path.basename(path_bvs_us_csv).split('_')[0]
    df = pd.read_csv(path_bvs_us_csv)
    filter_df = df[df['label'].isin(labels_us)]
    filter_sorted_df = filter_df.sort_values(by='poids', ascending=False)
    # BVS : attention il put y avoir plusieurs BVS pour un même modèle <=> plusieurs labels avec le même poids
    label_bvs_mesh = list(filter_sorted_df.loc[filter_sorted_df['poids'] ==  max(list(filter_sorted_df['poids']))]['label'])
    position_bvs_mesh = [[i for i in range(len(labels_us)) if labels_us[i] in label_bvs_mesh]]
    # coordonnées 3D des caméras BVS dans le repère de l'US
    cam_bvs = np.concatenate([np.array(X_us)[position_bvs_mesh], np.array(Y_us)[position_bvs_mesh], np.array(Z_us)[position_bvs_mesh]]).T  
    # Indice I-J des caméras dans le repère de l'US
    i_j_cam = np.concatenate([np.array(I_us)[position_bvs_mesh], np.array(J_us)[position_bvs_mesh]]).T
    all_bvs_us[name] = {'df': filter_sorted_df, 'label_bvs': label_bvs_mesh, 'cam_bvs': cam_bvs, 'ij_bvs': i_j_cam, 'poids_bvs' :  max(list(filter_sorted_df['poids']))}

In [8]:
all_bvs_us['carVasa']

{'df':    Unnamed: 0                  label  poids  sigma*100
 8           8     dessus_face_gauche   22.5       58.0
 2           2      dessus_face_droit   19.5       58.0
 1           1            dessus_face    9.8       58.0
 3           3    dessus_profil_droit    6.3       58.0
 7           7   dessus_profil_gauche    5.3       58.0
 5           5         dessus_arriere    4.2       58.0
 6           6  dessus_arriere_gauche    2.4       58.0
 4           4   dessus_arriere_droit    1.0       58.0,
 'label_bvs': ['dessus_face_gauche'],
 'cam_bvs': array([[ 1.1 ,  1.56, -1.1 ]]),
 'ij_bvs': array([[7., 1.]]),
 'poids_bvs': 22.500000000000004}

## Pour 1 categorie : cvs avec poids par caméras US

In [9]:
categorie_modelnet = 'car'
categorie_us = match_ModelNet2US[categorie_modelnet]; print(categorie_modelnet, categorie_us)

car carVasa


### Impact/Poids de la BVS de modelnet aligné sur les X cams US --> BVS sur l'ensemble des modèles 

In [10]:
## Fichiers BVS de la categorie
paths_bvs = [paths_bvs[i] for i in range(len(paths_bvs)) if categorie_modelnet in paths_bvs[i]]; print("Fichiers bvs de la categorie dispo :", len(paths_bvs), paths_bvs[0])
random.shuffle(paths_bvs)

Fichiers bvs de la categorie dispo : 38 car/train/car_0166_SMPLER_centered_scaled_remeshing_iso_iter4


Création des fichiers CAT_distribution_global_modelnet-Xcams.csv

In [11]:
all_bvs_modelnet = {}
## Df des poids de chaque mesh de la catégorie sur les caméras de l'US
df_poids_from_modelnet = pd.DataFrame(columns=['path', 'name']+[f"{int(I_us[k])}-{J_us[k]}" for k in range(len(I_us))])
df_poids_from_modelnet.loc[0] = [None, 'labels US'] + labels_us
# Pour chacun des modèles de la catégorie étudée dont on a le fichier bvs (que 38 actuellement), issue de ModelNet40 (car : 296)
for n in tqdm.tqdm(range(len(paths_bvs)*0)):
    ###################################################################
    ## ModelNet40
    # Load model alignés à l'étude utilisateur
    path_mesh_n = paths_bvs[n]; #print(path_mesh_n)
    name_modelnet_n =  '_'.join(os.path.basename(path_mesh_n).split('_')[:2])
    df_poids_from_modelnet.loc[n+1, 'path'] = path_mesh_n; df_poids_from_modelnet.loc[n+1, 'name'] = name_modelnet_n
    # Load mesh PRÉALABLEMENT placé dans le REPRÈRE US (avec code Alignenment/align_mesh.ipynb)
    path_mesh_modelnet_aligned_n = os.path.join(ModelNet40_aligned_us, path_mesh_n+"_aligned_ok_US.obj")
    mesh_modelnet_aligned_n = trimesh.load_mesh(path_mesh_modelnet_aligned_n)
    centroid_modelnet_aligned_n = get_centroid(mesh_modelnet_aligned_n.faces, mesh_modelnet_aligned_n.vertices)
    # BVS du mesh ModelNet40 aligned
    cams_modelnet_mesh_n, cam_bvs_modelnet_n, num_cam_bvs_modelnet_n = bvs_cams_modelnet_aligned(path_mesh_n, path_mesh_modelnet_aligned_n, dir_bvs, cams_modelnet)
    
    ## Objectif : trouver la "BVS moyenne"  <--> attribué un poids d'impact a chacun des cam de l'US
    # Impact de la camera BVS de modlenet40 sur les caméras de l'étude utilisateur
    df_poids_from_modelnet, cam_sphere = poids_modelnet_sur_US(df_poids_from_modelnet, cam_bvs_modelnet_n, centroid_modelnet_aligned_n, cams_us, R_sphere, I_us, J_us)

    ## OBJ : caméras modelent40 alignées avec US
    show_cams(mesh_modelnet_aligned_n, np.vstack((cam_bvs_modelnet_n, cam_sphere)), name_modelnet_n+"_sphere", None, None, None, dir_outputs_visu, US_obj=False)

    ###################################################################
    ## User stuy
    # mesh random from User_study
    path_mesh_us = os.path.join(dir_Data, categorie_us, categorie_us+"_update_normed_centered_user_study.obj"); #print(path_mesh_us)
    mesh_us = trimesh.load_mesh(path_mesh_us)
    # BVS US
    cam_bvs_us = all_bvs_us[categorie_us]['cam_bvs']
    ## Double obj : len(cams_us) cameras US et 12 caméras modelent40 alignées avec US
    show_cams(mesh_modelnet_aligned_n, cams_modelnet_mesh_n, name_modelnet_n, mesh_us, cams_us, categorie_us,  dir_outputs_visu)  
    
###################################################################
## Normalisation  des poids de modelnet sur US 
# Somme des poids pour chaque caméras
df_poids_from_modelnet_final = pd.DataFrame(columns=['label', 'poids'])
for k in range(len(I_us)*0):
    label_poids_k = list(df_poids_from_modelnet.loc[:, f"{int(I_us[k])}-{J_us[k]}"])
    df_poids_from_modelnet_final.loc[k] = [label_poids_k[0], np.sum(label_poids_k[1:])] 


###################################################################   
# ## BVS de la catégorie courante
df_poids_from_modelnet_final_sorted = df_poids_from_modelnet_final.sort_values(by='poids', ascending=False)
# # sauvegarde
#df_poids_from_modelnet_final_sorted.to_csv(os.path.join(dir_outputs_csv, categorie_modelnet+"_distribution_global_modelnet-"+str(len(X_us))+"cams.csv"))

0it [00:00, ?it/s]


## Score de proximité

In [12]:
# Dicitonnaire : values == categorie et keys == info sur les BVS : cam, label, ij ....
BVS = {}

# for categorie in list_categorie_us :
categorie = categorie_us; print(categorie)

## bvs modelnet 
df_bvs_modelnet = pd.read_csv(os.path.join(dir_outputs_csv, categorie_modelnet+"_distribution_global_modelnet-"+str(len(j_cam_us_ok)*8)+"cams.csv"))
label_bvs_modelnet = list(df_bvs_modelnet.loc[df_bvs_modelnet['poids'] ==  max(list(df_bvs_modelnet['poids']))]['label'])
position_bvs_modelenet = [[i for i in range(len(labels_us)) if labels_us[i] in label_bvs_modelnet]]
# coordonnées 3D des caméras BVS dans le repère de l'US
cam_bvs_modelnet = np.concatenate([np.array(X_us)[position_bvs_modelenet], np.array(Y_us)[position_bvs_modelenet], np.array(Z_us)[position_bvs_modelenet]]).T  
# Indice I-J des caméras dans le repère de l'US
ij_bvs_modelnet = np.concatenate([np.array(I_us)[position_bvs_modelenet], np.array(J_us)[position_bvs_modelenet]]).T
print('BVS modelenet dans repère US :\n', cam_bvs_modelnet, label_bvs_modelnet, ij_bvs_modelnet)

## bvs US
cam_bvs_us = all_bvs_us[categorie_us]['cam_bvs']
ij_bvs_us =  all_bvs_us[categorie_us]['ij_bvs']
print('US : \n', cam_bvs_us, all_bvs_us[categorie_us]['label_bvs'], all_bvs_us[categorie_us]['ij_bvs'])


## Distance entre les deux BVS

# Sauvegarde 
BVS[categorie] = {'modelnet': {'cam': cam_bvs_modelnet, 'label': label_bvs_modelnet, 'ij': ij_bvs_modelnet}, 'US': {'cam': cam_bvs_us, 'label': all_bvs_us[categorie_us]['label_bvs'], 'ij': all_bvs_us[categorie_us]['ij_bvs'], 'poids': all_bvs_us[categorie_us]['poids_bvs'], 'df' : all_bvs_us[categorie_us]['df']}}
BVS

carVasa
BVS modelenet dans repère US :
 [[-0.    1.56 -1.56]] ['dessus_profil_gauche'] [[6. 1.]]
US : 
 [[ 1.1   1.56 -1.1 ]] ['dessus_face_gauche'] [[7. 1.]]


{'carVasa': {'modelnet': {'cam': array([[-0.  ,  1.56, -1.56]]),
   'label': ['dessus_profil_gauche'],
   'ij': array([[6., 1.]])},
  'US': {'cam': array([[ 1.1 ,  1.56, -1.1 ]]),
   'label': ['dessus_face_gauche'],
   'ij': array([[7., 1.]]),
   'poids': 22.500000000000004,
   'df':    Unnamed: 0                  label  poids  sigma*100
   8           8     dessus_face_gauche   22.5       58.0
   2           2      dessus_face_droit   19.5       58.0
   1           1            dessus_face    9.8       58.0
   3           3    dessus_profil_droit    6.3       58.0
   7           7   dessus_profil_gauche    5.3       58.0
   5           5         dessus_arriere    4.2       58.0
   6           6  dessus_arriere_gauche    2.4       58.0
   4           4   dessus_arriere_droit    1.0       58.0}}}

proximity score

In [13]:
def score_proximite(BVS, categorie_us, list_meshs_sym, df_coords, sig=1.3, epsilon=2, scores_Ds = [0,0,1]):
    bvs_courant = BVS[categorie_us]; cam_pov_u_sym = None; symetrique = False
    ## bvs US
    ij_pov_u = list(BVS[categorie_us]['US']['ij'])[0]; cam_pov_u = np.around(list(BVS[categorie_us]['US']['cam'])[0], 2) ; print('pov_u', cam_pov_u)
    ## bvs Modelnet
    ij_pov_m = list(BVS[categorie_us]['modelnet']['ij'])[0]; cam_pov_m = np.around(list(BVS[categorie_us]['modelnet']['cam'])[0], 2) ; print('pov_m', cam_pov_m)
    label_pov_m = list(BVS[categorie_us]['modelnet']['label'])[0]
    #####################################################
    #### Distance sémantique DS de la catégroie courante
    ## Si les pov sont les mêmes
    if np.array_equal(ij_pov_u, ij_pov_m) : Ds = scores_Ds[-1]; print("cool c'est le meme pov")
    ## Categorie avec des objets NON symetriques
    elif not(categorie_us in list_meshs_sym):  Ds = get_ds2(cam_pov_u, cam_pov_m, sig, epsilon); print("cat pas sym")
    ## categorie symetrique 
    else : ## Est ce que pov_u a un symetrique
        sym_to_u = get_sym(ij_pov_u, df_coords); print("cat sym")
        if sym_to_u[0]: 
            print("pov a un sym")
            # symetrique de ij_pov_u
            ij_pov_u_sym = sym_to_u[1]
            cam_pov_u_sym = np.around(sym_to_u[2],2); print('sym', ij_pov_u_sym, cam_pov_u_sym)
            # si le pov_u_sym == pov de la methode
            if np.array_equal(ij_pov_u_sym, ij_pov_m) : Ds = scores_Ds[-1]; print("c'est les memes")
            ##S'il n'y a pas de symetrique au pov
            else : 
                Ds = max(get_ds2(cam_pov_u, cam_pov_m, sig, epsilon), 
                            get_ds2(cam_pov_u_sym, cam_pov_m, sig, epsilon)); print("mais pas les memes")
        ## Symetrique MAIS le pov_u n'a pas de symétique : ex : pov_u == Face pour les voitutres 
        else :  Ds = get_ds2(cam_pov_u, cam_pov_m, sig, epsilon); print("pov n'a pas de sym")

    #####################################################
    #### Impact -- Poids de la categorie courante
    Poids, _ = get_poids1(ij_pov_m, label_pov_m, bvs_courant['US'], df_coords)
    score =  max(Ds, Poids)
    print("Ds",Ds, "- W", Poids)
    
    ## Visualisation cam_u, cam_m, cam_u_sym
    path_mesh_us = os.path.join(dir_Data, categorie_us, categorie_us+"_update_normed_centered_user_study.obj"); #print(path_mesh_us)
    mesh_us = trimesh.load_mesh(path_mesh_us)
    cams = [cam_pov_u, cam_pov_m]; colors = [[1,0,1], [0,1,0]]
    if cam_pov_u_sym is not None : cams.append(cam_pov_u_sym); colors.append([0,0,1])
    show_some_cams(mesh_us, f"{categorie_us}_poriximity_score-{len(X_us)}cams", cams, colors, dir_outputs_visu)

    return score

In [14]:
score_proximite(BVS, categorie_us, list_categroie_us_sym, data_us_cam)

pov_u [ 1.1   1.56 -1.1 ]
pov_m [-0.    1.56 -1.56]
cat sym
pov a un sym
sym [1, 1] [1.1  1.56 1.1 ]
mais pas les memes
Ds 0.66 - W 0.27999999999999997


np.float64(0.66)

# Debug

In [ ]:
a

In [ ]:
# ## OBJ file with (12 cameras + obj)
# # Vertices and faces of the model user study
# verts_mesh_modelnet = []#np.array(mesh_modelnet.vertices)
# faces_mesh_modelnet = []#np.array(mesh_modelnet.faces) 
# # colors : Blanc cam 1 --> Jaune cam 4 ---> Rouge cam 10 --> Rouge foncé cam 12
# colormap = plt.get_cmap('hot'); colors_modelnet = colormap(np.linspace(0, 1, 12))[::-1] 
# #with open(os.path.join(dir_outputs, categorie_modelnet+"_modelNet40+12cam.obj"), 'w') as obj_file:
#     # Write vertices
#     for vertex in verts_mesh_modelnet:
#         obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]} 128 128 128\n")      
#     # Write 12 cameras positions
#     vertex_offset = len(verts_mesh_modelnet)
#     for cube_center, cube_color in zip(cams_modelnet_mesh[:, :3], colors_modelnet[:,:3]):  
#         vertex_offset = add_cube_to_obj(obj_file, cube_center, cube_color, vertex_offset, cube_size=0.3)
#     # Write faces
#     for face in faces_mesh_modelnet:
#         # Convert face indices to 1-based indexing
#         obj_file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")   
        
# ################################################################################"" 
# ## OBJ file with (8 cameras + obj)
# # Vertices and faces of the model user study
# verts_mesh_us = np.array(mesh_us.vertices)
# faces_mesh_us = np.array(mesh_us.faces) 
# # colors : Jaune cam 1 --> Vert cam 3 ---> Bleu cam 5 --> Bleu foncé cam 8
# colormap = plt.get_cmap('hot'); colors_us = colormap(np.linspace(0, 1, 8))[::-1] 
# # Write obj
# #with open(os.path.join(dir_outputs, categorie_us+"_us+8cam.obj"), 'w') as obj_file:
#     # Write vertices
#     for vertex in verts_mesh_us:
#         obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]} 128 128 128\n")      
#     # Write 8 cameras positions
#     vertex_offset = len(verts_mesh_us)
#     for cube_center, cube_color in zip(cams_us[:, :3], colors_us[:,:3]):  
#         vertex_offset = add_cube_to_obj(obj_file, cube_center, cube_color, vertex_offset, cube_size=0.3)
#     # Write faces
#     for face in faces_mesh_us:
#         # Convert face indices to 1-based indexing
#         obj_file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")   

### BVS : match celle de l'US et celle de ModelNet40